In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!git clone https://github.com/gangweiX/IGEV.git

In [ ]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch -c nvidia
!pip install opencv-python
!pip install scikit-image
!pip install tensorboard
!pip install matplotlib
!pip install tqdm
!pip install timm==0.5.4

In [ ]:
!python IGEV/IGEV-Stereo/demo_imgs.py --restore_ckpt ./sceneflow.pth -l IGEV/roses_L.png -r IGEV/roses_R.png

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

'''
Rotate left and right input images
'''
left_img = cv2.imread("/content/IGEV/roses_L.png")
rotated_left_img = left_img[:, ::-1]
# cv2_imshow(rotated_left_img)
cv2.imwrite("/content/IGEV/roses_L_rotated.png", rotated_left_img)

right_img = cv2.imread("/content/IGEV/roses_R.png")
rotated_right_img = right_img[:, ::-1]
# cv2_imshow(rotated_right_img)
cv2.imwrite("/content/IGEV/roses_R_rotated.png", rotated_right_img)


'''
Generate rotated version of right disparity map
'''
!python IGEV/IGEV-Stereo/demo_imgs.py --restore_ckpt ./sceneflow.pth -l /content/IGEV/roses_R_rotated.png -r /content/IGEV/roses_L_rotated.png
'''
Correct orientation of right disparity map
'''
right_disp_rotated = cv2.imread("/content/demo-output/IGEV.png")
# cv2_imshow(right_disp_rotated)
right_disp = right_disp_rotated[:, ::-1]
cv2_imshow(right_disp)
cv2.imwrite("/content/demo-output/roses_right_disparity.png", right_disp)